In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
scales = [8]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [7]:
# Verifier/ data consumer side:
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.geometric_mean(x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:171: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:121: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.abs((torch.log(self.result)*size)-torch.sum(torch.log(x)))<=size*torch.log(torch.tensor(1+self.error))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/on

In [8]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 8, param_scale: 8, scale_input_multiplier: 10) ------------->

+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error  | max_error | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+---------------+-----------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00054359436 | -0.0010871887 | 0         | -0.0010871887 | 0.00054359436  | 0.0010871887     | 0.0010871887  | 0             | 0.00000059098966   | -0.000011396554    | 0.000011396554         |
+----------------+---------------+-----------+---------------+----------------+--------

==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-1546,45056],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":6617,"total_const_size":1507,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},{"Ln":{"scale":256.0}},"Abs"],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709631663708}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 6.885586977005005 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.69921875
==== Generating Proof ====
proof:  {'instances': [['ad297b8184503f0131009e1b94795f879c4560a1c673ff814dc0228dafd72103', '0100000000000000000000000000000000000000000000000000000000000000', 'b32f000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x305ccb931c14ad928ce7463a7b1683647c74effb97989819b13a14aa8593aa7f168e9ac1715d2ac73fe439f2584f853dc60ca1d0c0f207041aabdb2c0e30cf1525537d500656ee426f0061f8e65c55779ee46bde2d1bd65aaa71027b3808eaf901721d79a97822e3e7dc34aee2098413ae06061d69fb4b21598a0b2f45b2b4b40190f93d8895b86e38fdb2455a675ddfeba055bca795ad70cfe17b5174f7840d07c3412f6bc50714b16257fd4d0241e33cd0e7c61339494862d2dbe71949c74f1caf0cea8e83bcfa69e2a317f12f5048a9fba76d4373405c391943903f7bf1401a28f29f6529d16b2b52c08a969ce8c53158902eadc335c138bb1049ef4a3390297ca8b25f9a07ab4dc92b41dca136cc5b035634b7e4691f90e5e6c5f87d8d9416de4b72f

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [47.69921875]
